In [1]:
import operator

class ScienceNerBaseline():
    
    # load the training set, and insert 2 'unknown' tokens between each paragraph
    def load(self,inFile) :
        txtfile = open(inFile, "r")
        self.tokens = []
        for i in txtfile :
            if i.strip() == "" :
                self.tokens.append(("<UNK>", "<UNK>"))
                self.tokens.append(("<UNK>", "<UNK>"))
                continue
            wd, tag = i.strip().split()
            self.tokens.append((wd, tag))
        txtfile.close()
        
    
    # this function slides a 3-word window over the entire training set word-by-word to
    # get every trigram, and classifies the 3-gram to be whatever tag the current word is
    def countTrigrams(self):
        self.counts = defaultdict(lambda: defaultdict(lambda: 0.0))
        self.context_totals = dict()
        w_1, w_2 = None, None
        for word in self.tokens:
            
            # word and tag
            wd, tk = word
            if w_1 is not None and w_2 is not None:
                self.counts[(w_2,w_1,wd)][tk] += 1
            w_2 = w_1
            w_1 = wd

    
    # this function run the test set and calculates accuracy
    def evaluate(self, testfile) :        
        txtfile = open(testfile, "r")
        testtokens = []
        for i in txtfile :
            
            # if there is a blank space in the test file, add 2 unknown tokens
            if i.strip() == "" :
                testtokens.append(("<UNK>", "<UNK>"))
                testtokens.append(("<UNK>", "<UNK>"))
                continue
            wd, tag = i.strip().split()
            testtokens.append((wd, tag))
        txtfile.close()
        
                
        w_1, w_2 = None, None
        num_correct = num_test = 0
        for word in testtokens:
            wd, tk = word
            
            # skip unknown words and create 2 dummy trigram tokens for beginning
            if w_1 is not None and w_2 is not None and wd is not "<UNK>":
                # context is w_2, w_1, wd
                num_test += 1
                prediction = max(self.counts[(w_2,w_1,wd)].iteritems())
                
                # correct prediction
                if prediction[0] == tk :
                    num_correct +=1
                    
            w_2 = w_1
            w_1 = wd
            
            
        accuracy = num_correct * 1.0 / num_test
        return accuracy
        

In [5]:
import os
from collections import defaultdict

modelInst = ScienceNerBaseline()

os.chdir('../data')
modelInst.load("test.txt")
modelInst.countTrigrams()
acc = modelInst.evaluate("test.txt")
print "Accuracy is :", acc

Accuracy is : 0.921436588103
